# Trajectory Structure Overview

This notebook documents the structure of a *single* recorded trajectory file so someone else can quickly understand what’s inside.

We’ll use `data/trajectory_20251215_143042.npz` as an example and inspect:
- Top-level keys and array shapes
- Core arrays (`timestamps`, `states`, `actions`)
- End-effector and ArUco pose data
- `metadata` contents and label conventions


In [3]:
import numpy as np
from pathlib import Path

# Change this path if you want to inspect a different file
TRAJ_PATH = Path("data/trajectory_20251215_143042.npz")

if not TRAJ_PATH.exists():
    raise FileNotFoundError(f"Trajectory file not found: {TRAJ_PATH}")

print(f"Loading trajectory: {TRAJ_PATH}")

data = np.load(TRAJ_PATH, allow_pickle=True)


Loading trajectory: data/trajectory_20251215_143042.npz


In [4]:
# Top-level keys and basic info

print("Available keys in .npz file:\n")
for key in data.keys():
    arr = data[key]
    if isinstance(arr, np.ndarray):
        print(f"  - {key:20s} shape={arr.shape}, dtype={arr.dtype}")
    else:
        print(f"  - {key:20s} type={type(arr).__name__}")

# Convenience handles
timestamps = data["timestamps"]
states = data["states"]
actions = data["actions"]
metadata = data["metadata"].item() if "metadata" in data else {}

print("\nSummary:")
print(f"  Num samples: {len(timestamps)}")
print(f"  Duration:   {timestamps[-1]:.2f} s")
print(f"  State shape: {states.shape}")
print(f"  Action shape: {actions.shape}")
if "ee_poses_debug" in data:
    print(f"  EE pose shape: {data['ee_poses_debug'].shape}")


Available keys in .npz file:

  - timestamps           shape=(404,), dtype=float64
  - states               shape=(404, 6), dtype=float64
  - actions              shape=(404, 7), dtype=float64
  - ee_poses_debug       shape=(404, 7), dtype=float64
  - object_pose          shape=(404, 7), dtype=float64
  - object_visible       shape=(404, 1), dtype=float64
  - aruco_ee_in_world    shape=(404, 7), dtype=float64
  - aruco_object_in_world shape=(404, 7), dtype=float64
  - aruco_ee_in_object   shape=(404, 7), dtype=float64
  - aruco_object_in_ee   shape=(404, 7), dtype=float64
  - aruco_visibility     shape=(404, 3), dtype=float64
  - augmented_actions    shape=(404, 10), dtype=float64
  - metadata             shape=(), dtype=object

Summary:
  Num samples: 404
  Duration:   20.17 s
  State shape: (404, 6)
  Action shape: (404, 7)
  EE pose shape: (404, 7)


In [5]:
# States and actions: shapes, labels, and first few rows

state_labels = metadata.get("state_labels", [f"state_{i}" for i in range(states.shape[1])])
action_labels = metadata.get("action_labels", [f"action_{i}" for i in range(actions.shape[1])])

print("State array (robot state):")
print(f"  shape:  {states.shape}")
print(f"  labels: {state_labels}")
print("  first 3 rows:\n", states[:3])

print("\nAction array (command / control):")
print(f"  shape:  {actions.shape}")
print(f"  labels: {action_labels}")
print("  first 3 rows:\n", actions[:3])


State array (robot state):
  shape:  (404, 6)
  labels: ['joint_0', 'joint_1', 'joint_2', 'joint_3', 'joint_4', 'gripper']
  first 3 rows:
 [[-0.01227484  0.61681087 -0.71347525 -0.8975979   0.05063373 -0.0252309 ]
 [-0.01227484  0.61681087 -0.71347525 -0.8975979   0.05063373 -0.0252309 ]
 [-0.01227484  0.61681087 -0.71347525 -0.8975979   0.05063373 -0.0252309 ]]

Action array (command / control):
  shape:  (404, 7)
  labels: ['vx', 'vy', 'vz', 'wx', 'wy', 'wz', 'gripper_target']
  first 3 rows:
 [[ 0.         0.         0.         0.         0.         0.
  -0.0252309]
 [ 0.         0.         0.         0.         0.         0.
  -0.0252309]
 [ 0.         0.         0.         0.         0.         0.
  -0.0252309]]


In [9]:
# Augmented action array (extra features for learning), if present

if "augmented_actions" in data:
    aug = data["augmented_actions"]
    aug_labels = metadata.get("augmented_action_labels", None)

    print("augmented_actions array:")
    print(f"  shape: {aug.shape}")
    if aug_labels is not None:
        print(f"  labels: {aug_labels}")
    else:
        print("  labels: (not provided in metadata; see code for definition)")
    print("  first 3 rows:\n", aug[:3])
else:
    print("No 'augmented_actions' array in this file.")


augmented_actions array:
  shape: (404, 10)
  labels: (not provided in metadata; see code for definition)
  first 3 rows:
 [[ 0.         0.         0.         0.         0.         0.
   0.         0.         0.        -0.0252309]
 [ 0.         0.         0.         0.         0.         0.
   0.         0.         0.        -0.0252309]
 [ 0.         0.         0.         0.         0.         0.
   0.         0.         0.        -0.0252309]]


In [6]:
# EE pose and ArUco-related arrays (if present)

if "ee_poses_debug" in data:
    ee = data["ee_poses_debug"]  # [N, 7] = [x, y, z, qw, qx, qy, qz]
    ee_labels = metadata.get("ee_pose_debug_labels", ["x", "y", "z", "qw", "qx", "qy", "qz"])
    print("EE poses (simulation frame):")
    print(f"  shape:  {ee.shape}")
    print(f"  labels: {ee_labels}")
    print("  first row:", ee[0])

aruco_keys = [
    "aruco_ee_in_world",
    "aruco_object_in_world",
    "aruco_ee_in_object",
    "aruco_object_in_ee",
    "aruco_visibility",
]

print("\nArUco-related keys present:")
for k in aruco_keys:
    if k in data:
        print(f"  - {k}: shape={data[k].shape}")
    else:
        print(f"  - {k}: (not present)")


EE poses (simulation frame):
  shape:  (404, 7)
  labels: ['ee_x', 'ee_y', 'ee_z', 'ee_qw', 'ee_qx', 'ee_qy', 'ee_qz']
  first row: [ 0.25432628  0.00262811  0.1446856   0.6562745  -0.20484254 -0.19713195
  0.69891509]

ArUco-related keys present:
  - aruco_ee_in_world: shape=(404, 7)
  - aruco_object_in_world: shape=(404, 7)
  - aruco_ee_in_object: shape=(404, 7)
  - aruco_object_in_ee: shape=(404, 7)
  - aruco_visibility: shape=(404, 3)


In [10]:
# Metadata dump: what extra info is stored alongside the arrays?

print("Metadata contents:\n")
for k, v in metadata.items():
    print(f"  {k}: {v}")

# Example: how someone might consume this file in their own code

example = {
    "timestamps": "1D array of length N (seconds since start)",
    "states": f"(N, {states.shape[1]}) float32 array, labels={state_labels}",
    "actions": f"(N, {actions.shape[1]}) float32 array, labels={action_labels}",
    "augmented_actions": "(N, K) extra action-related features for learning (see code / paper)",
    "ee_poses_debug": "(N, 7) [x, y, z, qw, qx, qy, qz] if present",
    "aruco": "world-frame and relative poses; see aruco_* keys above",
}

print("\nHigh-level summary for downstream users:\n")
for k, desc in example.items():
    print(f"- {k}: {desc}")


Metadata contents:

  num_samples: 404
  control_frequency: 20.0
  duration_seconds: 20.173153201001696
  state_dim: 6
  action_dim: 7
  ee_pose_debug_dim: 7
  state_labels: ['joint_0', 'joint_1', 'joint_2', 'joint_3', 'joint_4', 'gripper']
  action_labels: ['vx', 'vy', 'vz', 'wx', 'wy', 'wz', 'gripper_target']
  ee_pose_debug_labels: ['ee_x', 'ee_y', 'ee_z', 'ee_qw', 'ee_qx', 'ee_qy', 'ee_qz']
  ee_pose_debug_note: End-effector pose trajectory for debugging/inspection only, NOT used by policy
  extra_fields: ['object_pose', 'object_visible', 'aruco_ee_in_world', 'aruco_object_in_world', 'aruco_ee_in_object', 'aruco_object_in_ee', 'aruco_visibility', 'augmented_actions']
  timestamp: 2025-12-15T14:30:42.999847

High-level summary for downstream users:

- timestamps: 1D array of length N (seconds since start)
- states: (N, 6) float32 array, labels=['joint_0', 'joint_1', 'joint_2', 'joint_3', 'joint_4', 'gripper']
- actions: (N, 7) float32 array, labels=['vx', 'vy', 'vz', 'wx', 'wy', 'wz